In [3]:
search_for = 126
start_from = 0
threads = 10
thread = 4
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0023016929626464844
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 375
all cases: 956096141
type: [1, 1, 1, 1, 126] 126
cases of this type: 252047376
100000 24.90132402040347
200000 40.47524875520602
300000 49.243719140991494
400000 53.674775406203565
500000 53.43248861972335
600000 51.03824633714122
700000 50.68496823873081
800000 55.14308962286225
900000 58.81009476224909
1000000 45.21661605428924
1100000 38.590692731709396
1200000 43.91375118118818
1300000 49.23549849444272
1400000 53.29014798699045
1500000 54.93557011059626
1600000 56.18605894976723
1700000 61.95331590540789
1800000 45.63399007124542
1900000 67.11279353722158
2000000 42.16371386292643
2100000 27.063032207369936
2200000 49.93621206415051
2300000 51.529265111012776
2400000 56.84229559647431
2500000 54.10532394654264
2600000 62.84674324676789
2700000 49.92643363275448
2800000 36.58787490706881
2900000 39.56104258528098
3000000 30.042522442

30100000 75.790868513258
30200000 76.95554643577373
30300000 32.84998033825982
30400000 75.0174198846047
30500000 75.18821764923875
30600000 71.18517241237613
30700000 74.19209709617147
30800000 66.53803494053088
30900000 67.3552531684221
31000000 54.34229343002144
31100000 39.90924974069688
31200000 66.05558439672286
31300000 66.29916430431777
31400000 72.89530002455143
31500000 72.61561507153827
31600000 70.691343985054
31700000 74.20624287145196
31800000 42.044332588992
31900000 70.23174827729407
32000000 73.55290682866503
32100000 81.69398198542159
32200000 72.96559105425551
32300000 36.37129949174401
32400000 72.76677065902854
32500000 78.17521449086705
32600000 73.70732431112143
32700000 72.5556485957677
32800000 69.6456494362138
32900000 68.9438811135667
33000000 57.98949085529031
33100000 40.94545719320932
33200000 69.68354849634092
33300000 66.61021436341383
33400000 70.80674061977633
33500000 71.56362147332098
33600000 74.53227903681305
33700000 73.68804823447289
33800000 34.

60600000 53.42397748301752
60700000 64.17279037123063
60800000 65.59892548809755
60900000 68.84581650000108
61000000 54.68892317180388
61100000 35.4551932534062
61200000 65.76980982949338
61300000 66.80170431919497
61400000 63.35795723856337
61500000 65.19483756965063
61600000 34.05443300262157
61700000 68.69702604601554
61800000 72.01697918716017
61900000 71.37224025429153
62000000 75.42407470946281
62100000 76.63100816264986
62200000 71.17149249852177
62300000 71.00434250129359
62400000 71.06845636538574
62500000 57.61034714828378
62600000 40.09019052262173
62700000 60.92812887249603
62800000 63.20175628980625
62900000 63.16410681311665
63000000 56.48327140060297
63100000 35.999775239251726
63200000 62.49296752735325
63300000 66.17244259267483
63400000 64.24205751396781
63500000 57.73470811410803
63600000 38.48620277990357
63700000 67.68780047664389
63800000 72.69201779825282
63900000 69.44489456465493
64000000 74.82964050785162
64100000 76.91442632313404
64200000 71.63078662265407
6

90900000 61.27011580971391
91000000 55.60632197516367
91100000 30.725606946371883
91200000 59.229235250027365
91300000 50.072553613590685
91400000 43.33822473536514
91500000 57.72453897824647
91600000 60.76410997541798
91700000 60.920076779078784
91800000 65.81188339983512
91900000 64.00123439589858
92000000 65.3317711206229
92100000 66.79540364191728
92200000 69.24785719903514
92300000 66.71653462582879
92400000 66.16806360949639
92500000 62.59099788585768
92600000 63.18272889108636
92700000 63.20611142905674
92800000 28.54161185080856
92900000 61.68904125321938
93000000 53.580352095164415
93100000 30.958810809641705
93200000 61.794956091076635
93300000 44.12610920022771
93400000 52.14984080331576
93500000 61.016494608110406
93600000 61.79382926400144
93700000 66.3516739673343
93800000 66.15513605464629
93900000 65.79434867790135
94000000 68.80206952952267
94100000 67.42885064584554
94200000 66.87851209339001
94300000 68.23476020079947
94400000 63.70802849710329
94500000 64.5756410244

120400000 75.03388377284145
120500000 75.97211332819442
120600000 73.01541788235058
120700000 75.02610737621113
120800000 76.1047771049272
120900000 76.23573086196983
121000000 39.32630321123515
121100000 12.016930263347698
121200000 65.51227338844103
121300000 73.78473942527727
121400000 72.50544794201427
121500000 78.20096057310435
121600000 75.73017568835051
121700000 76.44555213757096
121800000 74.92518854827257
121900000 79.29084800595358
122000000 81.86199742428082
122100000 26.353989569436894
122200000 6.94089578039855
122300000 6.705092367053864
122400000 6.772427199841915
122500000 6.779398481802163
122600000 6.9485534603738905
122700000 6.603061973692236
122800000 6.7975070038424725
122900000 6.442305631781584
123000000 6.695228076579134
123100000 6.504076222120683
123200000 6.94021647336293
123300000 6.370785567090341
123400000 7.036685094581857
123500000 7.001516085952689
123600000 7.574755603717963
123700000 6.778214295643754
123800000 6.238409013533861
123900000 7.5884791

...

884500000 2.1274234445668783
884600000 3.348782945402277
884700000 2.444422448161782
884800000 3.4827166158934535
884900000 2.157396454103563
885000000 3.053984778939804
885100000 2.2918865237459847
885200000 3.021003140451783
885300000 3.0473568437486813
885400000 2.270626282997515
885500000 3.2094018136142672
885600000 2.0456244851835943
885700000 3.4085078169367304
885800000 1.684997378867483
885900000 3.2477335805696104
886000000 1.8034561146429553
886100000 2.9635976106040594
886200000 2.520881234158614
886300000 2.5619307197022203
886400000 3.930857025036317
886500000 2.4022842533066457
886600000 1.6503847693098987
886700000 0.5218832105503658
886800000 0.5225631296683743
886900000 0.5241210358785258
887000000 0.5267851565782965
887100000 0.5174986203250513
887200000 0.5264693300774674
887300000 0.5186739798097114
887400000 0.5223295354433232
887500000 0.5187007735804492
887600000 0.5172240130077342
887700000 2.7091562731798686
887800000 2.3715544895660696
887900000 4.13839882112

909200000 0.3533135360619724
best so far: 0
type: [1, 3, 6, 7, 1] 126
cases of this type: 37044
best so far: 0
type: [1, 3, 7, 1, 6] 126
cases of this type: 190512
909300000 0.9939861370836571
909400000 0.4671724959950447
best so far: 0
type: [1, 3, 7, 2, 3] 126
cases of this type: 95256
909500000 0.3575571535122335
best so far: 0
type: [1, 3, 7, 3, 2] 126
cases of this type: 63504
909600000 0.34062291989503707
best so far: 0
type: [1, 3, 7, 6, 1] 126
cases of this type: 31752
best so far: 0
type: [1, 3, 14, 1, 3] 126
cases of this type: 47628
909700000 0.33251620667790505
best so far: 0
type: [1, 3, 14, 3, 1] 126
cases of this type: 15876
best so far: 0
type: [1, 3, 21, 1, 2] 126
cases of this type: 21168
best so far: 0
type: [1, 3, 21, 2, 1] 126
cases of this type: 10584
best so far: 0
type: [1, 3, 42, 1, 1] 126
cases of this type: 5292
best so far: 0
type: [1, 6, 1, 1, 21] 126
cases of this type: 1166886
909800000 0.6275714897473355
909900000 0.9040948702738683
910000000 0.891251136

920800000 1.42665224515255
920900000 1.0252980576920152
921000000 1.2607432923108552
921100000 1.41414506457578
921200000 0.9095172758502305
921300000 1.3865265929181323
921400000 0.9623148257570949
921500000 1.261082012394929
921600000 1.4012845535167533
921700000 0.9256491891223668
921800000 1.4386339191024384
921900000 1.0959017192220053
922000000 1.40267990237041
922100000 0.41820530636297465
922200000 0.2573586028354479
922300000 0.2597761558209929
922400000 0.2617481587195476
922500000 0.2537522732915587
922600000 0.25397531334936485
922700000 0.25151341029048563
922800000 0.2530122641194092
922900000 0.2600356182349391
923000000 0.2591298494671941
923100000 0.2508747824123038
923200000 0.24548968103335964
923300000 0.7694912362006446
923400000 1.1605961156969513
923500000 1.4590394101702842
923600000 1.3601310000913323
923700000 0.8810913866787261
923800000 1.3001548041625672
923900000 0.9101226509600818
924000000 1.1190522723206966
924100000 1.3001349748932147
924200000 0.84799

944700000 0.21538429942435555
best so far: 0
type: [2, 3, 1, 3, 7] 126
cases of this type: 194481
944800000 0.18262787160626717
944900000 0.13808712736797135
best so far: 0
type: [2, 3, 1, 7, 3] 126
cases of this type: 83349
945000000 0.12467939353860814
best so far: 0
type: [2, 3, 1, 21, 1] 126
cases of this type: 27783
best so far: 0
type: [2, 3, 3, 1, 7] 126
cases of this type: 64827
945100000 0.08793952831021348
best so far: 0
type: [2, 3, 3, 7, 1] 126
cases of this type: 9261
best so far: 0
type: [2, 3, 7, 1, 3] 126
cases of this type: 11907
best so far: 0
type: [2, 3, 7, 3, 1] 126
cases of this type: 3969
best so far: 0
type: [2, 3, 21, 1, 1] 126
cases of this type: 1323
best so far: 0
type: [2, 7, 1, 1, 9] 126
cases of this type: 45927
best so far: 0
type: [2, 7, 1, 3, 3] 126
cases of this type: 15309
945200000 0.09321154704064329
best so far: 0
type: [2, 7, 1, 9, 1] 126
cases of this type: 5103
best so far: 0
type: [2, 7, 3, 1, 3] 126
cases of this type: 5103
best so far: 0
typ

955400000 0.00904963995501995
955500000 0.006696049519554112
best so far: 0
type: [6, 1, 1, 3, 7] 126
cases of this type: 64827
best so far: 0
type: [6, 1, 1, 7, 3] 126
cases of this type: 27783
955600000 0.005535589744902982
best so far: 0
type: [6, 1, 1, 21, 1] 126
cases of this type: 9261
best so far: 0
type: [6, 1, 3, 1, 7] 126
cases of this type: 21609
best so far: 0
type: [6, 1, 3, 7, 1] 126
cases of this type: 3087
best so far: 0
type: [6, 1, 7, 1, 3] 126
cases of this type: 3969
best so far: 0
type: [6, 1, 7, 3, 1] 126
cases of this type: 1323
best so far: 0
type: [6, 1, 21, 1, 1] 126
cases of this type: 441
best so far: 0
type: [6, 3, 1, 1, 7] 126
cases of this type: 7203
best so far: 0
type: [6, 3, 1, 7, 1] 126
cases of this type: 1029
best so far: 0
type: [6, 3, 7, 1, 1] 126
cases of this type: 147
best so far: 0
type: [6, 7, 1, 1, 3] 126
cases of this type: 567
best so far: 0
type: [6, 7, 1, 3, 1] 126
cases of this type: 189
best so far: 0
type: [6, 7, 3, 1, 1] 126
cases of